In [1]:
import sys
!{sys.executable} -m pip install datasets

In [3]:
import sys
!{sys.executable} -m pip install transformers

  Using cached transformers-4.17.0-py3-none-any.whl (3.8 MB)
  Using cached tokenizers-0.11.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.5 MB)
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

from transformers import  DataCollatorWithPadding

from transformers import  TrainingArguments, Trainer

import tensorflow as tf

In [5]:
raw_datasets = load_dataset("mrm8488/fake-news")

Using custom data configuration mrm8488--fake-news-8d5fa3c57f03ef12


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/111M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/mrm8488--fake-news-8d5fa3c57f03ef12/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 44898
    })
})

In [7]:
raw_train_dataset = raw_datasets["train"]
raw_train_dataset[0]

{'label': 1,
 'text': 'The Trump administration has been getting absolutely pummeled for its refusal to acknowledge climate change, even as it becomes more obvious in our surroundings.On Monday, CNN s Jim Acosta tore Trump s Homeland Security advisor, Tom Bossert, apart in front of everyone for the administration s ignorance. Acknowledging the fact that several massive hurricanes have been ravaging the United States, Acosta cornered Bossert, asking him if Trump and his administration  saw a connection between climate change and homeland security and that the frequency and intensity of powerful storms like Harvey and Irma can pose a problem for future administrations  and if the storms were  giving the administration some pause when it comes to the issue of climate change and homeland security. Bossert was almost beside himself as he said: Causality s something outside of my ability to analyze right now. We continue to take seriously the climate change, not the cause of it, but the thin

# Train Test Split

In [8]:
df_split=raw_train_dataset.train_test_split(test_size=0.2)

In [9]:
train= df_split["train"]
test= df_split["test"]

In [10]:
train

Dataset({
    features: ['text', 'label'],
    num_rows: 35918
})

In [11]:
test

Dataset({
    features: ['text', 'label'],
    num_rows: 8980
})

In [12]:
checkpoint = "ghanashyamvtatti/roberta-fake-news"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/560 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [13]:
train = train.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
train.set_format(type='torch', columns=['input_ids','attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(train, batch_size=32)

  0%|          | 0/36 [00:00<?, ?ba/s]

In [14]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
training_args = TrainingArguments("test-trainer", num_train_epochs=1,max_steps=100)

In [16]:
num_labels=2
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

In [18]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    tokenizer=tokenizer,  
)

max_steps is given, it will override any value given in num_train_epochs


In [19]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 35918
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 100


Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=100, training_loss=0.36805736541748046, metrics={'train_runtime': 157.7482, 'train_samples_per_second': 5.071, 'train_steps_per_second': 0.634, 'total_flos': 210488844288000.0, 'train_loss': 0.36805736541748046, 'epoch': 0.02})

In [20]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained('./test-trainer/')

Configuration saved in ./test-trainer/config.json
Model weights saved in ./test-trainer/pytorch_model.bin


In [21]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [22]:
test = test.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length'), batched=True)
test.set_format(type='torch', columns=['input_ids','attention_mask', 'label'])
dataloader = torch.utils.data.DataLoader(test, batch_size=32)

  0%|          | 0/9 [00:00<?, ?ba/s]

In [23]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=train,
    eval_dataset=test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
   
)


max_steps is given, it will override any value given in num_train_epochs


In [24]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 8980
  Batch size = 8


{'eval_accuracy': 0.9986636971046771,
 'eval_loss': 0.007348346523940563,
 'eval_runtime': 626.3333,
 'eval_samples_per_second': 14.337,
 'eval_steps_per_second': 1.793}